In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import datetime
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
md_zip_ranges = [
    (20331,20331),
    (20335,20797),
    (20812,21930)
]

md_cities = pd.DataFrame(
    {'City':      ['Baltimore','Annapolis','Hagerstown','Frederick', 'Ft. Meade',
                   'Salisbury', 'Ocean City', 'Bethesda', 'Cumberland', 'College Park'],
     'Latitude':  [39.289452,    38.978132,  39.642500,  39.4141457, 39.1038739, 
                   38.3654624, 38.332159, 38.984372, 39.648437, 38.9897],
     'Longitude': [-76.613124,  -76.488142, -77.718646, -77.4092184,-76.742847, 
                   -75.5983227, -75.087294, -77.094755, -78.762657, -76.9378]
    }
)

cities_df = gpd.GeoDataFrame(
    md_cities, geometry=gpd.points_from_xy(md_cities.Longitude, md_cities.Latitude))

md_zips = ['%05d' % zipc for zip_range in md_zip_ranges
                     for zipc in range(zip_range[0],zip_range[1]+1)]

cases = pd.DataFrame(
    [np.NaN for _ in range(len(md_zips))],
    index=md_zips,
    columns=['cases']
)


# MD County COVID-19 Cases By Zip Code

In [ ]:
zips_json = requests.get('https://services.arcgis.com/njFNhDsUCentVYJW/arcgis/rest/services/ZIPCodes_MD_1/FeatureServer/0/query?f=json&where=ProtectedCount%3E%3D8&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=*&orderByFields=ZIPCODE1%20asc&resultOffset=0&resultRecordCount=300&cacheHint=true')
current_cases_d = dict((x['attributes']['ZIPCODE1'],x['attributes']['ProtectedCount'])
                        for x in zips_json.json()['features'])

current_cases = pd.DataFrame(
    current_cases_d.values(),
    index=current_cases_d.keys(),
    columns=['cases'])
current_cases

In [ ]:
cases.update(current_cases)
cases[cases['cases'].notnull()]

Loading 2010 census data by zip code.  Data is broken down by age range and gender.  Summary rows have no gender.

population,minimum_age,maximum_age,gender,zipcode,geo_id

In [ ]:
pop = pd.read_csv('population_by_zip_2010.csv',dtype={'zipcode': str})
pop['ZIPCODE'] = pop['zipcode']
pop.set_index('ZIPCODE')

print(type(pop['ZIPCODE'].values[0]))
pop = pop[(pop['gender'] != 'male') & (pop['gender'] != 'female')]

Merge case data for Anne Arundel county with population

In [ ]:
pop_cases = cases.merge(pop,left_on=cases.index,right_on='ZIPCODE')
pop_cases.loc[pop_cases.ZIPCODE == '20740']

Read zip codes geo file and merge with population data

In [ ]:
zips = gpd.read_file('cb_2018_us_zcta510_500k.shp')
zips['ZIPCODE'] = zips['ZCTA5CE10']
zips.set_index('ZIPCODE')
zips = zips.merge(pop_cases,on='ZIPCODE',how='inner')
zips

# Cases per 1,000

In [ ]:
zips['density'] = zips['cases'] / (zips['population'] / 1_000)
zips.set_index('ZIPCODE')
print(zips[zips.density.notnull()][['ZIPCODE','density','cases','population']].sort_values('density').tail())

In [ ]:
print(zips[zips.zipcode == '20740']['geometry'])

In [ ]:
top5_zips = pd.DataFrame(
    {'City':      [],
     'Latitude':  [],
     'Longitude': []
    }
)

zip_names = dict((x['attributes']['ZIPCODE1'], x['attributes']['ZIPName'])
                        for x in zips_json.json()['features'])

for index, row in (zips[zips.density.notnull()][['ZIPCODE','density','cases','population']].sort_values('density').tail()).iterrows():
    city = zip_names[row['ZIPCODE']]
    point = zips[zips.zipcode == row['ZIPCODE']]['geometry'].centroid
    
    # this is an ugly hack to go back to lat and long
    point_str = str(point).split()
    long = float(point_str[2][1:])
    lat = float(point_str[3][:-1])
    #print ({'City':city, 'Latitude':lat, 'Longitude':long})
    top5_zips = top5_zips.append({'City':city, 'Latitude':lat, 'Longitude':long}, ignore_index=True) 
    
top5_zips_df = gpd.GeoDataFrame(
    top5_zips, geometry=gpd.points_from_xy(top5_zips.Longitude, top5_zips.Latitude))

print (top5_zips_df)


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(64,32))
divider = make_axes_locatable(ax)
cax = divider.append_axes("bottom", size="5%", pad=0.1)
zplot = zips.plot(column='density',
                  legend=True,
                  figsize=(16,12),
                  ax=ax, cax=cax,
                  legend_kwds={
                    'label': "Cases / 1000",
                    'orientation': "horizontal"  
                  },
                  missing_kwds={
                      "color": "lightgrey",
                      "label": "Missing values"
                  })


cities_plot = cities_df.plot(ax=zplot,marker='o',color='white',markersize=15)
_ = cities_df.apply(lambda x: cities_plot.annotate(s=x.City, 
                                                   xy=x.geometry.centroid.coords[0], 
                                                   ha='center', color='white'),
                    axis=1)

cities_plot = top5_zips_df.plot(ax=zplot,marker='o',color='red',markersize=10)
_ = top5_zips_df.apply(lambda x: cities_plot.annotate(s=x.City, 
                                                   xy=x.geometry.centroid.coords[0], 
                                                   ha='center', color='red'),
                    axis=1)
today = datetime.date.today().strftime('%Y-%m-%d')

plt.savefig('%s-MD-density.png' % today)